In [7]:
%pip install --upgrade openai
%pip install --upgrade typing-extensions

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Home

In [ ]:
from openai import OpenAI
import os

from dotenv import load_dotenv
load_dotenv()

## Set the API key and model name
MODEL="gpt-4o"
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as an env var>"))

In [ ]:
# Our OpenAPI call

# Feel free to modify the prompts as you see fit.



def check_image(client, model, image_path, system_prompt, user_prompt):
    completion = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": system_prompt}, # <-- This is the system message that provides context to the model
        {"role": "user", "content": [
            {"type": "text", "text": user_prompt},
            {"type": "image_url", "image_url": {
                "url": f"data:image/png;base64,{image_path}"}
            }
        ]}
    ],
    )
    return completion

In [ ]:
# Get the images from the folder. 

import os
import base64
import csv

def get_image_names(folder_path):
    # Supported image file extensions
    image_extensions = {'.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.webp'}

    # List to store image names
    image_names = []

    # Iterate over files in the given folder
    for filename in os.listdir(folder_path):
        # Check if the file has an image extension
        if os.path.splitext(filename)[1].lower() in image_extensions:
            image_names.append(filename)

    return image_names

def load_processed_combinations():
    processed = set()
    try:
        with open("results.csv", "r", newline='') as resultfile:
            reader = csv.DictReader(resultfile)
            for row in reader:
                processed.add((row['system_prompt'], row['user_prompt']))
    except FileNotFoundError:
        pass
    return processed

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

def ensure_csv_files_exist():
    # Ensure log.csv exists and has the correct header
    if not os.path.exists("log.csv"):
        with open("log.csv", "w", newline='') as logfile:
            csv.DictWriter(logfile, fieldnames=['system_prompt', 'user_prompt', 'folder', 'image_path', 'result', 'reason']).writeheader()

    # Ensure results.csv exists and has the correct header
    if not os.path.exists("results.csv"):
        with open("results.csv", "w", newline='') as resultfile:
            csv.DictWriter(resultfile, fieldnames=['system_prompt', 'user_prompt', 'folder', 'total_images', 'yes_count', 'no_count']).writeheader()

all_results = []

def run(system_prompt, user_prompt, folder_paths, processed_combinations):
    with open("log.csv", "a", newline='') as logfile, open("results.csv", "a", newline='') as resultfile:
        log_fieldnames = ['system_prompt', 'user_prompt', 'folder', 'image_path', 'result', 'reason']
        result_fieldnames = ['system_prompt', 'user_prompt', 'folder', 'total_images', 'yes_count', 'no_count']
        log_writer = csv.DictWriter(logfile, fieldnames=log_fieldnames)
        result_writer = csv.DictWriter(resultfile, fieldnames=result_fieldnames)

        for folder_path in folder_paths:
            image_names = get_image_names(folder_path)
            if (system_prompt, user_prompt) in processed_combinations:
                print("Skipping system/user prompt since they are already in results.csv")
                continue
            print(f"Processing images in the folder {folder_path}")

            yes_count = 0
            no_count = 0

            for name in image_names:
                image_path = os.path.join(folder_path, name)
                print(f"Processing image: {image_path}")

                encoded_image = encode_image(image_path)

                ans = check_image(client, MODEL, encoded_image, system_prompt, user_prompt).choices[0].message.content

                if ans[0] == 'Y':
                    yes_count += 1
                elif ans[0] == 'N':
                    no_count += 1

                log_writer.writerow({
                    'system_prompt': system_prompt,
                    'user_prompt': user_prompt,
                    'folder': folder_path,
                    'image_path': image_path,
                    'result': ans[0],
                    'reason': ans[1:]
                })

                print(f"Result for {image_path}: {ans}\n")

            total_images = len(image_names)
            result_writer.writerow({
                'system_prompt': system_prompt,
                'user_prompt': user_prompt,
                'folder': folder_path,
                'total_images': total_images,
                'yes_count': yes_count,
                'no_count': no_count
            })

            print(f"Summary for folder {folder_path}: {total_images} images processed, {yes_count} Yes, {no_count} No.\n")

        print(f"Processing complete for prompt: {system_prompt}")

def run_all_prompts(folder_paths):
    # Initialize CSV files with headers
    ensure_csv_files_exist()

    processed_combinations = load_processed_combinations()
    # Read prompts from prompt.csv and run for each prompt
    with open("prompt.csv", "r", newline='') as promptfile:
        prompt_reader = csv.DictReader(promptfile)
        for row in prompt_reader:
            system_prompt = row['system_prompt']
            user_prompt = row['user_prompt']
            run(system_prompt, user_prompt, folder_paths, processed_combinations)


In [ ]:
# Run

#
# system_prompt = "You are a human who has a keen eye for AI generated images. When prompted, answer YES or NO to the user's prompt"
# user_prompt = "Is this image generated or modified by AI at all?"

folder_paths = ["data/generated", "data/gt_background"]

#print(run(system_prompt, user_prompt, folder_paths)) # Manual method to run one prompt
print(run_all_prompts(folder_paths))

Processing images in the folder data/generated
Processing image: data/generated/IMG_2367.png
Result for data/generated/IMG_2367.png: N: Based on the visual evidence present in this image, there are no obvious signs that this image has been AI-generated or modified. The photograph displays consistent lighting, natural shadows, and realistic textures on the laptop and ID card, which are typical characteristics of a genuine photograph.

Processing image: data/generated/IMG_2366.png
Result for data/generated/IMG_2366.png: N: The image does not exhibit obvious characteristics of AI generation or modification. The depicted Arizona driver's license and the laptop keyboard appear natural and consistent with authentic physical objects. Features like lighting, shadows, and perspective align well with a real-world setting.

Processing image: data/generated/IMG_2358.png


KeyboardInterrupt: 